In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import lit, col,  to_number, format_number

spark = SparkSession.builder \
    .appName('Hadoop - Scaling Spark') \
        .config("spark.sql.catalogImplementation", "hive") \
            .getOrCreate()

+   Load file, dataframe and data view instances

In [2]:
df_taxes2013 = spark.read.csv('zipcodeagi13.csv', header=True)
df_taxes2013.createOrReplaceTempView('sql_taxes2013')

+ Format the fields - use ideal data types and rename columns on spark df and view

In [3]:
df_cleaned_taxes = df_taxes2013.select(
    'state',
    (col('zipcode')/10).cast('integer').alias('zipcode'),
    col('mars1').cast('int').alias('single_returns'),
    col('mars2').cast('int').alias('joint_returns'),
    col('numdep').cast('int'),
    col('A02650').cast('double').alias('total_income_amount'),
    col('A00300').alias('taxable_interest_amount'),
    col('a01000').cast('double').alias('net_capital_gains'),
    col('a00900').cast('double').alias('biz_net_income')
)
df_cleaned_taxes.show(5)

+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
|state|zipcode|single_returns|joint_returns|numdep|total_income_amount|taxable_interest_amount|net_capital_gains|biz_net_income|
+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
|   AL|      0|        488030|       122290|571240|        1.1444868E7|             77952.0000|          23583.0|      824487.0|
|   AL|      0|        195840|       155230|383240|        1.7810952E7|             81216.0000|          54639.0|      252768.0|
|   AL|      0|         72710|       146880|189340|        1.6070153E7|             80627.0000|          84137.0|      259836.0|
|   AL|      0|         24860|       126480|134370|        1.4288572E7|             71086.0000|         105947.0|      214668.0|
|   AL|      0|         16930|       168170|177800|         2.605392E7|            149150.0000|  

In [4]:
sqlqry = '''
  SELECT 
    state, 
    cast(zipcode / 10 as int) zipcode,
    cast(mars1 as int) single_returns,
    int(mars2) as joint_returns,
    int(numdep) as numdep,
    cast(A02650 as decimal(10,2)) as total_income_amount,
    cast(A00300 as decimal(10,2)) as taxable_interest_amount,
    cast(a01000 as decimal(10,2)) as net_capital_gains,
    cast(a00900 as decimal(10,2)) as biz_net_income
  FROM sql_taxes2013
'''
spark.sql(sqlqry).createOrReplaceTempView('sql_cleaned_taxes')
spark.sql('select * from sql_cleaned_taxes').show(5)


+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
|state|zipcode|single_returns|joint_returns|numdep|total_income_amount|taxable_interest_amount|net_capital_gains|biz_net_income|
+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
|   AL|      0|        488030|       122290|571240|        11444868.00|               77952.00|         23583.00|     824487.00|
|   AL|      0|        195840|       155230|383240|        17810952.00|               81216.00|         54639.00|     252768.00|
|   AL|      0|         72710|       146880|189340|        16070153.00|               80627.00|         84137.00|     259836.00|
|   AL|      0|         24860|       126480|134370|        14288572.00|               71086.00|        105947.00|     214668.00|
|   AL|      0|         16930|       168170|177800|        26053920.00|              149150.00|  

 
+ Which states have the highest income per zip code?

In [9]:
df_cleaned_taxes.groupBy('state','zipcode').sum('biz_net_income')\
   .select('state','zipcode', col('sum(biz_net_income)').alias('NetIncome').cast('decimal'))\
    .orderBy('NetIncome', ascending = False).show(5)

+-----+-------+---------+
|state|zipcode|NetIncome|
+-----+-------+---------+
|   CA|      0| 51876045|
|   TX|      0| 29345002|
|   NY|      0| 24952005|
|   FL|      0| 16519558|
|   PA|      0| 12207777|
+-----+-------+---------+
only showing top 5 rows



In [6]:
spark.sql(
    '''
        select 
            state, zipcode, sum(biz_net_income) as NetIncome
        from sql_cleaned_taxes
        group by state, zipcode
        order by NetIncome desc
    '''
).show(5)

+-----+-------+-----------+
|state|zipcode|  NetIncome|
+-----+-------+-----------+
|   CA|      0|51876045.00|
|   TX|      0|29345002.00|
|   NY|      0|24952005.00|
|   FL|      0|16519558.00|
|   PA|      0|12207777.00|
+-----+-------+-----------+
only showing top 5 rows



+   Next two scripts performs same action in different ways

In [7]:
spark.sql(
''' 
    SELECT state,zipcode, SUM(net_capital_gains) AS cap_gains
    FROM sql_cleaned_taxes
    WHERE NOT (zipcode = 0000 OR zipcode = 9999)
    GROUP BY state,zipcode
    ORDER BY cap_gains ASC
'''
).show(2,truncate=False)

+-----+-------+---------+
|state|zipcode|cap_gains|
+-----+-------+---------+
|WV   |2659   |-20.00   |
|KY   |4126   |-14.00   |
+-----+-------+---------+
only showing top 2 rows



In [8]:
df_cleaned_taxes.filter((col('zipcode') != '0000') | (col('zipcode') != '9999')) \
    .groupBy('state','zipcode').sum('net_capital_gains') \
        .select('state','zipcode', col('sum(net_capital_gains)').alias('cap_gains')) \
            .orderBy('cap_gains').show(2)

+-----+-------+---------+
|state|zipcode|cap_gains|
+-----+-------+---------+
|   WV|   2659|    -20.0|
|   KY|   4126|    -14.0|
+-----+-------+---------+
only showing top 2 rows

